In [ ]:
import re
import pandas as pd
from pprint import pprint
import string

# NLTK
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import webtext
from nltk.probability import FreqDist
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(["the", "jul", "pm", "https", "www", "printable", "version", "thehindu", "com", "news", "ece", 
               "s", "t", "d", "m", "article", "b", "rb","cwe", "click", "subscribe", "photo", "credit", "said", "topvpn",
               "right", "caused"])

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Spacy
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Imporing json files
import json
from google.colab import files
uploaded = files.upload()

Saving thehindu_jan_11_file_1.json to thehindu_jan_11_file_1.json
Saving thehindu_jan_11_file_2.json to thehindu_jan_11_file_2.json
Saving thehindu_jan_11_file_3.json to thehindu_jan_11_file_3.json
Saving thehindu_jan_11_file_4.json to thehindu_jan_11_file_4.json
Saving thehindu_jan_11_file_5.json to thehindu_jan_11_file_5.json
Saving thehindu_jan_11_file_6.json to thehindu_jan_11_file_6.json
Saving thehindu_jan_11_file_7.json to thehindu_jan_11_file_7.json
Saving thehindu_jan_11_file_10.json to thehindu_jan_11_file_10.json
Saving thehindu_jan_11_file_12.json to thehindu_jan_11_file_12.json
Saving thehindu_jan_11_file_13.json to thehindu_jan_11_file_13.json
Saving thehindu_jan_11_file_14.json to thehindu_jan_11_file_14.json
Saving thehindu_jan_11_file_15.json to thehindu_jan_11_file_15.json
Saving thehindu_jan_11_file_16.json to thehindu_jan_11_file_16.json
Saving thehindu_jan_11_file_17.json to thehindu_jan_11_file_17.json
Saving thehindu_jan_11_file_18.json to thehindu_jan_11_file_18

In [ ]:
json_files = list(uploaded.keys())
json_files

['thehindu_jan_11_file_1.json',
 'thehindu_jan_11_file_2.json',
 'thehindu_jan_11_file_3.json',
 'thehindu_jan_11_file_4.json',
 'thehindu_jan_11_file_5.json',
 'thehindu_jan_11_file_6.json',
 'thehindu_jan_11_file_7.json',
 'thehindu_jan_11_file_10.json',
 'thehindu_jan_11_file_12.json',
 'thehindu_jan_11_file_13.json',
 'thehindu_jan_11_file_14.json',
 'thehindu_jan_11_file_15.json',
 'thehindu_jan_11_file_16.json',
 'thehindu_jan_11_file_17.json',
 'thehindu_jan_11_file_18.json',
 'thehindu_jan_11_file_19.json',
 'thehindu_jan_11_file_21.json',
 'thehindu_jan_11_file_23.json',
 'thehindu_jan_11_file_24.json',
 'thehindu_jan_11_file_27.json',
 'thehindu_jan_11_file_28.json',
 'thehindu_jan_11_file_29.json',
 'thehindu_jan_11_file_30.json',
 'thehindu_jan_11_file_31.json',
 'thehindu_jan_11_file_32.json',
 'thehindu_jan_11_file_33.json',
 'thehindu_jan_11_file_34.json',
 'thehindu_jan_11_file_35.json',
 'thehindu_jan_11_file_36.json',
 'thehindu_jan_11_file_37.json',
 'thehindu_jan_11

In [ ]:
corpus = []
for file_name in json_files:
  decoded_json = json.loads(uploaded[file_name].decode("utf-8"))
  corpus.append(''.join(decoded_json['text']))
corpus[0]

'\nDivyansh pips Rudrankksh for top spot\n\nKavi Rakshna takes the honours in the women’s air rifle\nWorld No. 1 in men’s air rifle, Divyansh Singh Panwar asserted his class by rising to the top position in the second competition of the National shooting selection trials on Sunday.Qualifiying in the eighth place with 628.7 points, a mere 0.1 point ahead of Srinjoy Datta, Divyansh shot 250.9 in the final, to beat Rudrankksh Patil by 0.8 point for the top spot.Hazarika thirdWorld junior champion Hriday Hazarika, who had won the first trial, was pushed to the third spot. Youth Olympics silver medallist Shahu Tushar Mane placed fourth while qualification topper Arjun Babuta (632.1) finished seventh.In the women’s section, Kavi Rakshna of Tamil Nadu emerged on top with 251.4, as she beat qualification topper Nisha Kanwar (630.7) by 0.7 point. Remarkably, Kavi had risen to the top spot after having placed seventh in qualification. World No. 1 and winner of first trial, Elavenil Valarivan pla

In [ ]:
df = pd.DataFrame(corpus, columns = {'Raw_Text'})
df

,Raw_Text
0,\nDivyansh pips Rudrankksh for top spot\n\nKav...
1,\nLombardy proves her class in HRC Golconda 10...
2,"\nVictory Parade, Race For The Stars, Demerara..."
3,\nLombardy for HRC Golconda 1000 Guineas The ...
4,\nISRO to adopt 100 Atal Tinkering Labs to pro...
...,...
326,\nData | MSMEs continue to be most stressed se...
327,"\nAmong the countries that began inoculation, ..."
328,\nKrish Ashok’s non-cookbook for kitchen nerds...
329,"\nRenowned writer Ved Mehta, who took India to..."


**Data Cleaning**

In [ ]:
# Removing extra whitespaces (Removes line characters too)
def remove_extra_whitespace(text):
  return re.sub("\s+"," ", text)

# Remove punctuations and convert to lower case
def lower_without_punct(text):
  lower_no_punct = "".join([i.lower() for i in text if i not in string.punctuation])
  words_wo_punct = ''.join(lower_no_punct)
  return words_wo_punct

def remove_numbers(text):
  return ''.join([i for i in text if not i.isdigit()])

cleaned_corpus = []
for doc in corpus:
  wo_extra_ws = remove_extra_whitespace(doc)
  lower_wo_punct = lower_without_punct(wo_extra_ws)
  wo_numbers = remove_numbers(lower_wo_punct)
  cleaned_corpus.append(wo_numbers)

cleaned_corpus[0]

' divyansh pips rudrankksh for top spot kavi rakshna takes the honours in the women’s air rifle world no  in men’s air rifle divyansh singh panwar asserted his class by rising to the top position in the second competition of the national shooting selection trials on sundayqualifiying in the eighth place with  points a mere  point ahead of srinjoy datta divyansh shot  in the final to beat rudrankksh patil by  point for the top spothazarika thirdworld junior champion hriday hazarika who had won the first trial was pushed to the third spot youth olympics silver medallist shahu tushar mane placed fourth while qualification topper arjun babuta  finished seventhin the women’s section kavi rakshna of tamil nadu emerged on top with  as she beat qualification topper nisha kanwar  by  point remarkably kavi had risen to the top spot after having placed seventh in qualification world no  and winner of first trial elavenil valarivan placed thirdin the second skeet trials amrinder singh cheema pippe

In [ ]:
df['Cleaned_Text'] = cleaned_corpus
df

,Raw_Text,Cleaned_Text
0,\nDivyansh pips Rudrankksh for top spot\n\nKav...,divyansh pips rudrankksh for top spot kavi ra...
1,\nLombardy proves her class in HRC Golconda 10...,lombardy proves her class in hrc golconda gu...
2,"\nVictory Parade, Race For The Stars, Demerara...",victory parade race for the stars demerara pi...
3,\nLombardy for HRC Golconda 1000 Guineas The ...,lombardy for hrc golconda guineas the fourye...
4,\nISRO to adopt 100 Atal Tinkering Labs to pro...,isro to adopt atal tinkering labs to promote...
...,...,...
326,\nData | MSMEs continue to be most stressed se...,data msmes continue to be most stressed sect...
327,"\nAmong the countries that began inoculation, ...",among the countries that began inoculation th...
328,\nKrish Ashok’s non-cookbook for kitchen nerds...,krish ashok’s noncookbook for kitchen nerds l...
329,"\nRenowned writer Ved Mehta, who took India to...",renowned writer ved mehta who took india to a...


**Tokenization and removing names of persons**

In [ ]:
corpus_wo_persons = []
all_person_list = []
nlp = en_core_web_sm.load()
for doc in cleaned_corpus:
  text = nlp(doc)
  tokens = []
  person_list = []

  for x in text.ents:
    if (x.label_=='PERSON'):
      person_list = person_list + x.text.split()

  wo_persons = [word for word in doc.split() if word not in person_list]
  all_person_list = all_person_list + person_list
  corpus_wo_persons.append(wo_persons)     

In [ ]:
all_person_list

['srinjoy',
 'datta',
 'divyansh',
 'hazarika',
 'fatehbir',
 'singh',
 'topper',
 'anantjeet',
 'singh',
 'naruka',
 'gurjoat',
 'singh',
 'kanwar',
 'elavenil',
 'valarivan',
 'fatehbir',
 'singh',
 'gurjoat',
 'singh',
 'areeba',
 'khan',
 'areeba',
 'khan',
 'mor',
 'afroz',
 'khan',
 'thp',
 'fp',
 'tanala',
 'reddy',
 'gollapudi',
 'm',
 'maiden',
 'yo',
 'darc',
 'akshay',
 'kumar',
 'thp',
 'fp',
 'tanala',
 'ram',
 'h',
 'nk',
 'm',
 's',
 'thp',
 'fp',
 'tanala',
 'rajini',
 'akshay',
 'kumar',
 'max',
 'afroz',
 'khan',
 'thp',
 'fp',
 'tanala',
 'donald',
 'anthony',
 'netto',
 'akshay',
 'kumar',
 'vita',
 'gaurav',
 'singh',
 'thp',
 'fp',
 'tanala',
 'laxman',
 'singh',
 'hrc',
 'golconda',
 'guineas',
 'gr',
 'mor',
 'thp',
 'fp',
 'tanala',
 'lvr',
 'br',
 'kumar',
 'abhay',
 'singh',
 'thp',
 'fp',
 'tanala',
 'ramaswamy',
 'thp',
 'fp',
 'tanala',
 'jakkula',
 'veerathu',
 'santaiah',
 'tkts',
 'ii',
 'tkts',
 'iii',
 'tkts',
 'ii',
 'kamigallu',
 'wellm',
 'demerara

In [ ]:
corpus_wo_persons

[['pips',
  'rudrankksh',
  'for',
  'top',
  'spot',
  'kavi',
  'rakshna',
  'takes',
  'the',
  'honours',
  'in',
  'the',
  'women’s',
  'air',
  'rifle',
  'world',
  'no',
  'in',
  'men’s',
  'air',
  'rifle',
  'panwar',
  'asserted',
  'his',
  'class',
  'by',
  'rising',
  'to',
  'the',
  'top',
  'position',
  'in',
  'the',
  'second',
  'competition',
  'of',
  'the',
  'national',
  'shooting',
  'selection',
  'trials',
  'on',
  'sundayqualifiying',
  'in',
  'the',
  'eighth',
  'place',
  'with',
  'points',
  'a',
  'mere',
  'point',
  'ahead',
  'of',
  'shot',
  'in',
  'the',
  'final',
  'to',
  'beat',
  'rudrankksh',
  'patil',
  'by',
  'point',
  'for',
  'the',
  'top',
  'spothazarika',
  'thirdworld',
  'junior',
  'champion',
  'hriday',
  'who',
  'had',
  'won',
  'the',
  'first',
  'trial',
  'was',
  'pushed',
  'to',
  'the',
  'third',
  'spot',
  'youth',
  'olympics',
  'silver',
  'medallist',
  'shahu',
  'tushar',
  'mane',
  'placed',
  '

In [ ]:
# Build the bigram models by providing the corpus and hyperparameters
bigram = gensim.models.Phrases(corpus_wo_persons, min_count=5, threshold=100) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

# Define functions for stopwords, bigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(corpus_wo_persons)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

df['Processed_Text'] = data_words_bigrams
df

,Raw_Text,Cleaned_Text,Processed_Text
0,\nDivyansh pips Rudrankksh for top spot\n\nKav...,divyansh pips rudrankksh for top spot kavi ra...,"[pips, rudrankksh, top, spot, kavi, rakshna, t..."
1,\nLombardy proves her class in HRC Golconda 10...,lombardy proves her class in hrc golconda gu...,"[lombardy, proves, class, trainer, deshmukh, c..."
2,"\nVictory Parade, Race For The Stars, Demerara...",victory parade race for the stars demerara pi...,"[victory, parade, race, stars, pirate, love, s..."
3,\nLombardy for HRC Golconda 1000 Guineas The ...,lombardy for hrc golconda guineas the fourye...,"[lombardy, guineas, fouryearold, filly, lombar..."
4,\nISRO to adopt 100 Atal Tinkering Labs to pro...,isro to adopt atal tinkering labs to promote...,"[isro, adopt, atal, tinkering, labs, promote, ..."
...,...,...,...
326,\nData | MSMEs continue to be most stressed se...,data msmes continue to be most stressed sect...,"[data, msmes, continue, stressed, sector, urba..."
327,"\nAmong the countries that began inoculation, ...",among the countries that began inoculation th...,"[among, countries, began, inoculation, pfizerb..."
328,\nKrish Ashok’s non-cookbook for kitchen nerds...,krish ashok’s noncookbook for kitchen nerds l...,"[krish, ashok, noncookbook, kitchen, nerds, lo..."
329,"\nRenowned writer Ved Mehta, who took India to...",renowned writer ved mehta who took india to a...,"[renowned, writer, took, india, americans, die..."


In [ ]:
from nltk.stem import WordNetLemmatizer  
lemmatizer = WordNetLemmatizer()

for m in range(len(json_files)):
  temp1 = []
  temp2 = df['Processed_Text'].values[m]
  for i in temp2:
    temp1.append(lemmatizer.lemmatize(i))
  df['Processed_Text'].values[m] = temp1 
df

,Raw_Text,Cleaned_Text,Processed_Text
0,\nDivyansh pips Rudrankksh for top spot\n\nKav...,divyansh pips rudrankksh for top spot kavi ra...,"[pip, rudrankksh, top, spot, kavi, rakshna, ta..."
1,\nLombardy proves her class in HRC Golconda 10...,lombardy proves her class in hrc golconda gu...,"[lombardy, prof, class, trainer, deshmukh, cla..."
2,"\nVictory Parade, Race For The Stars, Demerara...",victory parade race for the stars demerara pi...,"[victory, parade, race, star, pirate, love, sa..."
3,\nLombardy for HRC Golconda 1000 Guineas The ...,lombardy for hrc golconda guineas the fourye...,"[lombardy, guinea, fouryearold, filly, lombard..."
4,\nISRO to adopt 100 Atal Tinkering Labs to pro...,isro to adopt atal tinkering labs to promote...,"[isro, adopt, atal, tinkering, lab, promote, s..."
...,...,...,...
326,\nData | MSMEs continue to be most stressed se...,data msmes continue to be most stressed sect...,"[data, msmes, continue, stressed, sector, urba..."
327,"\nAmong the countries that began inoculation, ...",among the countries that began inoculation th...,"[among, country, began, inoculation, pfizerbio..."
328,\nKrish Ashok’s non-cookbook for kitchen nerds...,krish ashok’s noncookbook for kitchen nerds l...,"[krish, ashok, noncookbook, kitchen, nerd, loo..."
329,"\nRenowned writer Ved Mehta, who took India to...",renowned writer ved mehta who took india to a...,"[renowned, writer, took, india, american, dy, ..."


**Removing words that are smaller than 3**

In [ ]:
for i in range(len(json_files)):
  temp1 = []
  temp2 = df['Processed_Text'].values[i]
  for j in temp2:
    if (len(j)>2):
      temp1.append(j)
  df['Processed_Text'].values[i] = temp1 
df

,Raw_Text,Cleaned_Text,Processed_Text
0,\nDivyansh pips Rudrankksh for top spot\n\nKav...,divyansh pips rudrankksh for top spot kavi ra...,"[pip, rudrankksh, top, spot, kavi, rakshna, ta..."
1,\nLombardy proves her class in HRC Golconda 10...,lombardy proves her class in hrc golconda gu...,"[lombardy, prof, class, trainer, deshmukh, cla..."
2,"\nVictory Parade, Race For The Stars, Demerara...",victory parade race for the stars demerara pi...,"[victory, parade, race, star, pirate, love, sa..."
3,\nLombardy for HRC Golconda 1000 Guineas The ...,lombardy for hrc golconda guineas the fourye...,"[lombardy, guinea, fouryearold, filly, lombard..."
4,\nISRO to adopt 100 Atal Tinkering Labs to pro...,isro to adopt atal tinkering labs to promote...,"[isro, adopt, atal, tinkering, lab, promote, s..."
...,...,...,...
326,\nData | MSMEs continue to be most stressed se...,data msmes continue to be most stressed sect...,"[data, msmes, continue, stressed, sector, urba..."
327,"\nAmong the countries that began inoculation, ...",among the countries that began inoculation th...,"[among, country, began, inoculation, pfizerbio..."
328,\nKrish Ashok’s non-cookbook for kitchen nerds...,krish ashok’s noncookbook for kitchen nerds l...,"[krish, ashok, noncookbook, kitchen, nerd, loo..."
329,"\nRenowned writer Ved Mehta, who took India to...",renowned writer ved mehta who took india to a...,"[renowned, writer, took, india, american, staf..."


In [ ]:
# Deleting Cleaned_Text column from dataframe
del df['Cleaned_Text']
df

,Raw_Text,Processed_Text
0,\nDivyansh pips Rudrankksh for top spot\n\nKav...,"[pip, rudrankksh, top, spot, kavi, rakshna, ta..."
1,\nLombardy proves her class in HRC Golconda 10...,"[lombardy, prof, class, trainer, deshmukh, cla..."
2,"\nVictory Parade, Race For The Stars, Demerara...","[victory, parade, race, star, pirate, love, sa..."
3,\nLombardy for HRC Golconda 1000 Guineas The ...,"[lombardy, guinea, fouryearold, filly, lombard..."
4,\nISRO to adopt 100 Atal Tinkering Labs to pro...,"[isro, adopt, atal, tinkering, lab, promote, s..."
...,...,...
326,\nData | MSMEs continue to be most stressed se...,"[data, msmes, continue, stressed, sector, urba..."
327,"\nAmong the countries that began inoculation, ...","[among, country, began, inoculation, pfizerbio..."
328,\nKrish Ashok’s non-cookbook for kitchen nerds...,"[krish, ashok, noncookbook, kitchen, nerd, loo..."
329,"\nRenowned writer Ved Mehta, who took India to...","[renowned, writer, took, india, american, staf..."


In [ ]:
df.to_csv("Jan11.csv",index = True, header=True)